In [16]:
import pandas as pd
import numpy as np


df=pd.read_csv('data/math-syllabus.csv',low_memory=False)
df.loc[df['UnitTitle']=='Counting and Cardinality'].to_csv('Counting and Cardinality.csv')

In [22]:
#consolidate 

from pathlib import Path
import pandas as pd
from anthropic import Anthropic
import time
from datetime import datetime
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get API key from environment
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
if not ANTHROPIC_API_KEY:
    raise ValueError("ANTHROPIC_API_KEY not found in environment variables")

client = Anthropic(api_key=ANTHROPIC_API_KEY)

def refactor_skill_group(client: Anthropic, skill_data: pd.DataFrame) -> list:
    """
    Use Claude to refactor a group of related subskills into a more consolidated set
    """
    # Get skill info from first row
    skill_info = skill_data.iloc[0]
    
    # Create context string with current subskills
    subskills_text = "\n".join([
        f"- {row['SubskillDescription']} (Difficulty: {row['DifficultyStart']}-{row['DifficultyEnd']}, Target: {row['TargetDifficulty']})"
        for _, row in skill_data.iterrows()
    ])
    
    prompt = f"""Please analyze and refactor these kindergarten curriculum subskills to create a more streamlined and progressive set of learning objectives.

Current Skill Group:
Subject: {skill_info['Subject']}
Unit: {skill_info['UnitTitle']} ({skill_info['UnitID']})
Skill: {skill_info['SkillDescription']} ({skill_info['SkillID']})

Current subskills:
{subskills_text}

Please create a refactored set of 8-15 subskills that:
1. Eliminates redundancy while maintaining comprehensive coverage
2. Shows clear progression from basic to advanced
3. Combines similar concepts into unified skills
4. Maintains appropriate difficulty ranges for kindergarten

For each refactored subskill, provide EXACTLY in this format:
SUBSKILL_ID: {skill_info['SkillID']}-[A through H]
DESCRIPTION: [Clear, specific learning objective]
DIFFICULTY: [start]-[end]-[target]
---

Rules:
- Keep difficulty numbers between 1-10
- Target difficulty should be between start and end
- Each subskill should build on previous ones
- Descriptions should be specific and measurable
- Make sure progression is logical"""

    try:
        print("  • Sending request to Claude API...")
        response = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=2048,
            temperature=0.7,
            messages=[{"role": "user", "content": prompt}]
        )
        print("  • Received response from Claude")
        
        # Parse response into rows
        rows = []
        current_subskill = {}
        
        print("  • Parsing response...")
        for line in response.content[0].text.strip().split('\n'):
            line = line.strip()
            if not line or line == '---':
                if current_subskill and len(current_subskill) == 3:
                    # Parse difficulty string
                    diff_parts = current_subskill['DIFFICULTY'].split('-')
                    if len(diff_parts) == 3:
                        try:
                            rows.append({
                                'Subject': skill_info['Subject'],
                                'Grade': skill_info['Grade'],
                                'UnitID': skill_info['UnitID'],
                                'UnitTitle': skill_info['UnitTitle'],
                                'SkillID': skill_info['SkillID'],
                                'SkillDescription': skill_info['SkillDescription'],
                                'SubskillID': current_subskill['SUBSKILL_ID'],
                                'SubskillDescription': current_subskill['DESCRIPTION'],
                                'DifficultyStart': int(diff_parts[0]),
                                'DifficultyEnd': int(diff_parts[1]),
                                'TargetDifficulty': int(diff_parts[2])
                            })
                        except ValueError:
                            print(f"  ⚠️ Warning: Invalid difficulty values in response")
                current_subskill = {}
                continue
                
            if ': ' in line:
                key, value = line.split(': ', 1)
                current_subskill[key] = value.strip()
        
        print(f"  • Successfully parsed {len(rows)} refactored subskills")
        return rows
        
    except Exception as e:
        print(f"❌ Error refactoring skill {skill_info['SkillID']}: {str(e)}")
        return []

def refactor_curriculum(input_csv: str, output_csv: str):
    """
    Refactor entire curriculum by processing each skill group
    """
    start_time = time.time()
    print(f"\n🔄 Starting curriculum refactoring at {datetime.now().strftime('%H:%M:%S')}")
    
    # Read input CSV
    print(f"  • Reading {input_csv}")
    df = pd.read_csv(input_csv)
    print(f"  • Found {len(df)} total rows")
    
    # Group by SkillID
    skill_groups = df.groupby('SkillID')
    print(f"  • Identified {len(skill_groups)} skill groups to refactor")
    
    # Process each skill group
    all_refactored = []
    
    for idx, (skill_id, skill_data) in enumerate(skill_groups, 1):
        print(f"\n✨ Processing skill group {idx}/{len(skill_groups)}")
        print(f"  • Skill: {skill_id}: {skill_data.iloc[0]['SkillDescription']}")
        print(f"  • Current subskills: {len(skill_data)}")
        
        refactored = refactor_skill_group(client, skill_data)
        
        if refactored:
            all_refactored.extend(refactored)
            print(f"  • Successfully refactored into {len(refactored)} subskills")
            
            # Save progress
            temp_df = pd.DataFrame(all_refactored)
            temp_df.to_csv(f"{output_csv}.temp", index=False)
            print(f"  • Progress saved to temporary file")
        else:
            print(f"⚠️ Warning: No refactored skills generated for {skill_id}")
        
        # Add delay between API calls
        print("  • Waiting 2 seconds before next API call...")
        time.sleep(2)
    
    # Save final results
    if all_refactored:
        print("\n💾 Saving final results...")
        final_df = pd.DataFrame(all_refactored)
        final_df.to_csv(output_csv, index=False)
        print(f"✅ Saved {len(final_df)} total refactored subskills to {output_csv}")
        
        # Remove temp file
        Path(f"{output_csv}.temp").unlink(missing_ok=True)
        print("  • Temporary file removed")
        
        # Print statistics
        print("\n📊 Refactoring Statistics:")
        print(f"  • Original subskills: {len(df)}")
        print(f"  • Refactored subskills: {len(final_df)}")
        print(f"  • Reduction: {((len(df) - len(final_df)) / len(df) * 100):.1f}%")
        
        elapsed_time = time.time() - start_time
        print(f"\n🏁 Processing completed in {elapsed_time:.1f} seconds")
        
        return final_df
    else:
        print("\n❌ Error: No refactored skills were generated!")
        return None
    


input_csv = "data/language-arts-syllabus.csv"
output_csv = "language-arts_refactored.csv"

df = refactor_curriculum(input_csv, output_csv)


🔄 Starting curriculum refactoring at 22:25:03
  • Reading data/language-arts-syllabus.csv
  • Found 329 total rows
  • Identified 27 skill groups to refactor

✨ Processing skill group 1/27
  • Skill: LA001-01: Print Concepts
  • Current subskills: 14
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response...
  • Successfully parsed 9 refactored subskills
  • Successfully refactored into 9 subskills
  • Progress saved to temporary file
  • Waiting 2 seconds before next API call...

✨ Processing skill group 2/27
  • Skill: LA001-02: Phonological Awareness
  • Current subskills: 14
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response...
  • Successfully parsed 0 refactored subskills
⚠️ Warning: No refactored skills generated for LA001-02
  • Waiting 2 seconds before next API call...

✨ Processing skill group 3/27
  • Skill: LA001-03: Phonics and Word Recognition
  • Current subskills: 15
  • Sending request to Cla

In [5]:
from pathlib import Path
import pandas as pd
from anthropic import Anthropic
import time
from datetime import datetime
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get API key from environment
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
if not ANTHROPIC_API_KEY:
    raise ValueError("ANTHROPIC_API_KEY not found in environment variables")

client = Anthropic(api_key=ANTHROPIC_API_KEY)

def generate_detailed_objectives_for_subskill(subskill_data):
    """Generate detailed learning objectives for a specific subskill"""
    
    # Print debug info
    subskill_desc = subskill_data['SubskillDescription'].iloc[0]
    print(f"\n📝 Processing subskill: {subskill_desc}")
    
    # Create detailed context string from subskill data
    row = subskill_data.iloc[0]
    context = (f"Subject: {row['Subject']}\n"
              f"Unit: {row['UnitTitle']} ({row['UnitID']})\n"
              f"Skill: {row['SkillDescription']} ({row['SkillID']})\n"
              f"Subskill: {row['SubskillDescription']} ({row['SubskillID']})\n")

    prompt = f"""Please analyze this specific kindergarten curriculum subskill and generate 5-8 detailed learning objectives. Each objective should be on a new line, following the exact CSV format specified below.

{context}

The output must be EXACTLY in this CSV format (one objective per line):
{row['Subject']},{row['SubskillID']},{row['SubskillDescription']},<ConceptGroup>,<DetailedObjective>

For example:
Mathematics,{row['SubskillID']},{row['SubskillDescription']},Number Recognition,Match spoken numbers with written numerals
Mathematics,{row['SubskillID']},{row['SubskillDescription']},Counting Skills,Count objects sequentially from 1 to 10

Requirements for objectives:
1. Must be specific, measurable, and appropriate for kindergarten (age 5-6)
2. Show progression from basic to more advanced understanding
3. Include various contexts and real-world applications
4. Focus on this specific subskill's scope
5. Each line must follow the exact CSV format shown above

Generate at least 5 varied objectives that cover different aspects of learning this skill."""

    try:
        print("  • Sending request to Claude API...")
        response = client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=2048,
            temperature=0.7,
            messages=[{"role": "user", "content": prompt}]
        )
        print("  • Received response from Claude")
        
        # Print raw response for debugging
        print("\nRaw response from Claude:")
        print(response.content[0].text.strip())
        print("\nProcessing response lines...")
        
        # Parse and validate each line
        objectives = []
        for line in response.content[0].text.strip().split('\n'):
            if not line:
                continue
                
            try:
                # Split only on first two commas to get subject and subskill_id
                parts = line.split(',', 2)
                if len(parts) < 3:
                    print(f"  • Skipping line with insufficient parts: {line}")
                    continue
                
                subject, subskill_id, remainder = parts
                
                # Find the last two commas in the remainder to extract concept_group and objective
                last_comma_pos = remainder.rindex(',')
                second_last_comma_pos = remainder.rindex(',', 0, last_comma_pos)
                
                subskill_desc = remainder[:second_last_comma_pos]
                concept_group = remainder[second_last_comma_pos + 1:last_comma_pos].strip()
                objective = remainder[last_comma_pos + 1:].strip()
                
                # Validate all parts exist
                if not all([subject, subskill_id, subskill_desc, concept_group, objective]):
                    print("  • Skipping line with empty fields")
                    continue
                
                # Validate that SubskillID matches our input
                if subskill_id.strip() == row['SubskillID']:
                    objectives.append({
                        'Subject': subject.strip(),
                        'SubskillID': subskill_id.strip(),
                        'SubskillDescription': subskill_desc.strip(),
                        'ConceptGroup': concept_group,
                        'DetailedObjective': objective
                    })
                    print(f"  • Added objective: {concept_group} - {objective}")
                
            except ValueError as e:
                print(f"  • Error parsing line: {str(e)}")
                continue
        
        print(f"  • Successfully generated {len(objectives)} objectives")
        return objectives
        
    except Exception as e:
        print(f"❌ Error generating objectives for {row['SubskillID']}: {str(e)}")
        print(f"Exception details: {type(e).__name__}")
        import traceback
        traceback.print_exc()
        return []

def process_curriculum_from_csv(input_csv: str, output_csv: str, subject: str = None):
    """Process curriculum from CSV file and generate detailed objectives"""
    
    start_time = time.time()
    print(f"\n🔄 Starting processing at {datetime.now().strftime('%H:%M:%S')}")
    
    # Read CSV file
    print(f"  • Reading input CSV: {input_csv}")
    df = pd.read_csv(input_csv)
    
    # Filter by subject if specified
    if subject:
        df = df[df['Subject'] == subject]
        print(f"  • Filtered to {len(df)} rows for subject: {subject}")
    else:
        print(f"  • Processing all {len(df)} rows")
    
    all_objectives = []
    
    # Process each unique subskill
    unique_subskills = df['SubskillID'].unique()
    print(f"  • Found {len(unique_subskills)} unique subskills to process")
    
    try:
        for idx, subskill_id in enumerate(unique_subskills, 1):
            print(f"\n✨ Processing subskill {idx}/{len(unique_subskills)}")
            subskill_df = df[df['SubskillID'] == subskill_id]
            
            objectives = generate_detailed_objectives_for_subskill(subskill_df)
            if objectives:
                all_objectives.extend(objectives)
                
                # Save progress after each subskill
                temp_df = pd.DataFrame(all_objectives)
                temp_df.to_csv(f"{output_csv}.temp", index=False)
                print(f"  • Saved progress ({len(all_objectives)} total objectives so far)")
            else:
                print(f"⚠️ WARNING: No valid objectives generated for {subskill_id}")
            
            # Add a small delay between API calls
            print("  • Waiting 2 seconds before next API call...")
            time.sleep(2)
    except KeyboardInterrupt:
        print("\n⚠️ Processing interrupted by user")
    except Exception as e:
        print(f"\n❌ Error during processing: {str(e)}")
        import traceback
        traceback.print_exc()
    finally:
        # Save whatever we have so far
        if all_objectives:
            print("\n💾 Saving final results...")
            objectives_df = pd.DataFrame(all_objectives)
            objectives_df.to_csv(output_csv, index=False)
            print(f"✅ Saved {len(objectives_df)} total objectives to {output_csv}")
            
            # Remove temp file if it exists
            temp_file = Path(f"{output_csv}.temp")
            if temp_file.exists():
                temp_file.unlink()
        else:
            print("\n❌ WARNING: No objectives were generated!")
        
        elapsed_time = time.time() - start_time
        print(f"\n🏁 Processing completed in {elapsed_time:.1f} seconds")
        
        return objectives_df if all_objectives else None

input_csv = "data/science_refactored-syllabus.csv"
output_csv = "data/detailed_objectives_science.csv"
    
# Process all subjects
df = process_curriculum_from_csv(input_csv, output_csv)



🔄 Starting processing at 11:57:14
  • Reading input CSV: data/science_refactored-syllabus.csv
  • Processing all 88 rows
  • Found 88 unique subskills to process

✨ Processing subskill 1/88

📝 Processing subskill: Identify and demonstrate basic movements (push, pull, roll, slide) of everyday objects
  • Sending request to Claude API...
  • Received response from Claude

Raw response from Claude:
Science,SCI001-01-A,Identify and demonstrate basic movements (push, pull, roll, slide) of everyday objects,Movement Identification,Verbally name and point to at least 3 different types of object movements (push, pull, roll, slide)
Science,SCI001-01-A,Identify and demonstrate basic movements (push, pull, roll, slide) of everyday objects,Physical Demonstration,Physically demonstrate a push or pull motion with classroom toys and manipulatives
Science,SCI001-01-A,Identify and demonstrate basic movements (push, pull, roll, slide) of everyday objects,Surface Interaction,Predict and explain how diffe

In [7]:
from anthropic import Anthropic
import pandas as pd
from typing import List, Dict
import time
from pathlib import Path
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get API key from environment
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
if not ANTHROPIC_API_KEY:
    raise ValueError("ANTHROPIC_API_KEY not found in environment variables")

client = Anthropic(api_key=ANTHROPIC_API_KEY)

def expand_existing_skill(client: Anthropic, subject: str, grade: str, unit_id: str, unit_title: str, 
                         skill_id: str, skill_name: str, existing_subskills: list) -> list:
    """
    Expand an existing skill with additional subskills, returning rows for CSV format
    """
    print(f"\n📝 Generating new subskills for {skill_id}: {skill_name}")
    print(f"  • Found {len(existing_subskills)} existing subskills")
    print("  • Preparing prompt for Claude...")
    
    prompt = f"""Create 10-15 NEW specific, measurable learning objectives (subskills) for this kindergarten subject.
DO NOT duplicate any existing subskills.

Skill: {skill_name}

Existing subskills (DO NOT DUPLICATE THESE):
{chr(10).join('- ' + s for s in existing_subskills)}

Each new subskill should:
- Be distinct from existing subskills
- Be achievable in 1-2 learning sessions
- Build progressively in complexity
- Focus on observable outcomes
- Include clear success criteria

Format each NEW subskill exactly as follows:

SUBSKILL_ID: [skill_id-E] (continuing after existing A-D with E, F, etc.)
DESCRIPTION: [Clear, specific learning objective]
DIFFICULTY: [start 1-10]-[end 1-10]-[target 1-10]
---"""

    print("  • Sending request to Claude API...")
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        temperature=0.7,
        messages=[{"role": "user", "content": prompt}]
    )
    print("  • Received response from Claude")
    
    # Parse response into CSV rows
    rows = []
    current_subskill = {}
    
    print("  • Parsing response into CSV format...")
    for line in response.content[0].text.strip().split('\n'):
        line = line.strip()
        if not line or line == '---':
            if current_subskill and len(current_subskill) == 3:  # Only if we have all required fields
                # Parse difficulty string
                diff_parts = current_subskill['DIFFICULTY'].split('-')
                if len(diff_parts) == 3:
                    rows.append({
                        'Subject': subject,
                        'Grade': grade,
                        'UnitID': unit_id,
                        'UnitTitle': unit_title,
                        'SkillID': skill_id,
                        'SkillDescription': skill_name,
                        'SubskillID': current_subskill['SUBSKILL_ID'],
                        'SubskillDescription': current_subskill['DESCRIPTION'],
                        'DifficultyStart': diff_parts[0],
                        'DifficultyEnd': diff_parts[1],
                        'TargetDifficulty': diff_parts[2]
                    })
            current_subskill = {}
            continue
            
        if ': ' in line:
            key, value = line.split(': ', 1)
            current_subskill[key] = value.strip()
    
    # Process final subskill if exists
    if current_subskill and len(current_subskill) == 3:
        diff_parts = current_subskill['DIFFICULTY'].split('-')
        if len(diff_parts) == 3:
            rows.append({
                'Subject': subject,
                'Grade': grade,
                'UnitID': unit_id,
                'UnitTitle': unit_title,
                'SkillID': skill_id,
                'SkillDescription': skill_name,
                'SubskillID': current_subskill['SUBSKILL_ID'],
                'SubskillDescription': current_subskill['DESCRIPTION'],
                'DifficultyStart': diff_parts[0],
                'DifficultyEnd': diff_parts[1],
                'TargetDifficulty': diff_parts[2]
            })
    
    print(f"  • Successfully generated {len(rows)} new subskills")
    return rows

def expand_all_skills(client: Anthropic, input_csv: str, output_csv: str):
    """
    Expand all skills from input CSV and write directly to output CSV
    """
    print(f"\n🔄 Reading input CSV: {input_csv}")
    # Read existing data
    df = pd.read_csv(input_csv)
    print(f"  • Found {len(df)} total rows")
    
    # Create output dataframe with same columns
    output_rows = []
    
    # Get unique skills
    skills_df = df.drop_duplicates(['SkillID', 'SkillDescription'])
    print(f"  • Identified {len(skills_df)} unique skills to process")
    
    for idx, skill in skills_df.iterrows():
        skill_id = skill['SkillID']
        print(f"\n✨ Processing skill {idx + 1}/{len(skills_df)}")
        print(f"  • Current skill: {skill_id}: {skill['SkillDescription']}")
        
        # Get existing subskills for this skill
        skill_df = df[df['SkillID'] == skill_id]
        existing_subskills = skill_df['SubskillDescription'].tolist()
        
        # Expand skill
        new_rows = expand_existing_skill(
            client=client,
            subject=skill['Subject'],
            grade=skill['Grade'],
            unit_id=skill['UnitID'],
            unit_title=skill['UnitTitle'],
            skill_id=skill_id,
            skill_name=skill['SkillDescription'],
            existing_subskills=existing_subskills
        )
        
        output_rows.extend(new_rows)
        
        # Add delay between API calls
        print("  • Waiting 2 seconds before next API call...")
        time.sleep(2)
    
    print("\n📊 Combining results...")
    # Combine existing and new rows
    all_rows = pd.concat([df, pd.DataFrame(output_rows)])
    
    # Save to CSV
    print(f"💾 Saving results to {output_csv}")
    all_rows.to_csv(output_csv, index=False)
    print(f"✅ Complete! Final statistics:")
    print(f"  • Original rows: {len(df)}")
    print(f"  • New rows added: {len(output_rows)}")
    print(f"  • Total rows in output: {len(all_rows)}")

expand_all_skills(
        client=client,
        input_csv="data/science-syllabus.csv",
        output_csv="data/sciencesyllabus_expanded.csv"
    )


🔄 Reading input CSV: data/science-syllabus.csv
  • Found 39 total rows
  • Identified 11 unique skills to process

✨ Processing skill 1/11
  • Current skill: SCI001-01: Forces and Motion

📝 Generating new subskills for SCI001-01: Forces and Motion
  • Found 4 existing subskills
  • Preparing prompt for Claude...
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response into CSV format...
  • Successfully generated 12 new subskills
  • Waiting 2 seconds before next API call...

✨ Processing skill 5/11
  • Current skill: SCI001-02: Matter and Its Properties

📝 Generating new subskills for SCI001-02: Matter and Its Properties
  • Found 4 existing subskills
  • Preparing prompt for Claude...
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response into CSV format...
  • Successfully generated 12 new subskills
  • Waiting 2 seconds before next API call...

✨ Processing skill 9/11
  • Current skill: SCI001-03: Energy

📝 Ge

In [8]:
expand_all_skills(
        client=client,
        input_csv="data/social-studies-syllabus.csv",
        output_csv="data/ssocial-studies-syllabus_expanded.csv"
    )


🔄 Reading input CSV: data/social-studies-syllabus.csv
  • Found 38 total rows
  • Identified 11 unique skills to process

✨ Processing skill 1/11
  • Current skill: SS001-01: Rules and Responsibilities

📝 Generating new subskills for SS001-01: Rules and Responsibilities
  • Found 4 existing subskills
  • Preparing prompt for Claude...
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response into CSV format...
  • Successfully generated 12 new subskills
  • Waiting 2 seconds before next API call...

✨ Processing skill 5/11
  • Current skill: SS001-02: Citizenship

📝 Generating new subskills for SS001-02: Citizenship
  • Found 4 existing subskills
  • Preparing prompt for Claude...
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response into CSV format...
  • Successfully generated 12 new subskills
  • Waiting 2 seconds before next API call...

✨ Processing skill 9/11
  • Current skill: SS001-03: Government

📝 Genera

In [9]:
expand_all_skills(
        client=client,
        input_csv="data/language-arts-syllabus.csv",
        output_csv="data/language-arts-syllabus_expanded.csv"
    )


🔄 Reading input CSV: data/language-arts-syllabus.csv
  • Found 20 total rows
  • Identified 7 unique skills to process

✨ Processing skill 1/7
  • Current skill: LA001-01: Print Concepts

📝 Generating new subskills for LA001-01: Print Concepts
  • Found 3 existing subskills
  • Preparing prompt for Claude...
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response into CSV format...
  • Successfully generated 11 new subskills
  • Waiting 2 seconds before next API call...

✨ Processing skill 4/7
  • Current skill: LA001-02: Phonological Awareness

📝 Generating new subskills for LA001-02: Phonological Awareness
  • Found 3 existing subskills
  • Preparing prompt for Claude...
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response into CSV format...
  • Successfully generated 11 new subskills
  • Waiting 2 seconds before next API call...

✨ Processing skill 7/7
  • Current skill: LA001-03: Phonics and Word Recognitio

In [10]:
expand_all_skills(
        client=client,
        input_csv="data/kindergartenmathsyllabus v2.csv",
        output_csv="data/kindergartenmathsyllabus v2_expanded.csv"
    )


🔄 Reading input CSV: data/kindergartenmathsyllabus v2.csv
  • Found 56 total rows
  • Identified 12 unique skills to process

✨ Processing skill 1/12
  • Current skill: COUNT001-01: Know number names and the count sequence

📝 Generating new subskills for COUNT001-01: Know number names and the count sequence
  • Found 9 existing subskills
  • Preparing prompt for Claude...
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response into CSV format...
  • Successfully generated 12 new subskills
  • Waiting 2 seconds before next API call...

✨ Processing skill 10/12
  • Current skill: COUNT001-02: Count to tell the number of objects

📝 Generating new subskills for COUNT001-02: Count to tell the number of objects
  • Found 5 existing subskills
  • Preparing prompt for Claude...
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response into CSV format...
  • Successfully generated 11 new subskills
  • Waiting 2 seconds befor

In [11]:
expand_all_skills(
        client=client,
        input_csv="data/Arts-syllabus.csv",
        output_csv="data/Arts-syllabus_expanded.csv"
    )


🔄 Reading input CSV: data/Arts-syllabus.csv
  • Found 27 total rows
  • Identified 9 unique skills to process

✨ Processing skill 1/9
  • Current skill: ART001-01: Creating

📝 Generating new subskills for ART001-01: Creating
  • Found 3 existing subskills
  • Preparing prompt for Claude...
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response into CSV format...
  • Successfully generated 12 new subskills
  • Waiting 2 seconds before next API call...

✨ Processing skill 4/9
  • Current skill: ART001-02: Presenting

📝 Generating new subskills for ART001-02: Presenting
  • Found 3 existing subskills
  • Preparing prompt for Claude...
  • Sending request to Claude API...
  • Received response from Claude
  • Parsing response into CSV format...
  • Successfully generated 12 new subskills
  • Waiting 2 seconds before next API call...

✨ Processing skill 7/9
  • Current skill: ART001-03: Responding

📝 Generating new subskills for ART001-03: Responding
  